Trabalho 1 - Introdução ao Aprendizado de Máquina
Sérgio Hiroshi Honda - DRE:120035741

Importação das bibliotecas necessárias ao longo do código

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import pearsonr

Criação da função responsável por criar o arquivo de respostas e escrever cada ID e seu respectivo resultado previsto

In [2]:
def criar_resposta(arr):
    with open('arquivo_respostas.csv', 'w') as arq:
        arq.write('id_solicitante,inadimplente\n')
        for n in range(1, 5001):
            arq.write(f'{n + 20000},{arr[n - 1]}\n')

Importando os arquivos .csv dos dados de treino e de teste em forma de DataFrames, e retirando a primeira linha de ambos (Chaves Primárias)

In [3]:
dados_treinamento = pd.read_csv("conjunto_de_treinamento.csv")
dados_teste = pd.read_csv("conjunto_de_teste.csv")

dados_treinamento = dados_treinamento.iloc[:,1:]
dados_teste = dados_teste.iloc[:,1:]

Lista as colunas a serem removidas para aumentar a precisão do modelo (foi feita um loop, onde, ao retira-las, a acurácia do modelo melhorou)

In [4]:
colunas_removidas = [
    'grau_instrucao',
    'possui_telefone_celular',
    'meses_no_trabalho',
    'grau_instrucao_companheiro',
    'possui_email',
    'nacionalidade',
    'possui_telefone_celular'
]

Transformando as colunas que possuem valores de texto em colunas categóricas (númericas), para poderem ser trabalhadas pelo KNN. Além disso, remove as colunas encontradas anteriormente que pioram a qualidade do modelo.

In [5]:
colunas_texto = ['forma_envio_solicitacao', 'sexo', 'possui_telefone_residencial', 'vinculo_formal_com_empresa', 'possui_telefone_trabalho']
colunas_texto = set(colunas_texto) - set(colunas_removidas)

dados_treinamento_clean = pd.get_dummies(dados_treinamento, columns=colunas_texto)

dados_treinamento_clean.drop(columns=colunas_removidas, inplace=True)

C:\Users\sergi\AppData\Local\Temp\ipykernel_21388\2223330473.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dados_treinamento_clean = pd.get_dummies(dados_treinamento, columns=colunas_texto)


In [6]:
estados = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO',
           'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI',
           'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']

dados_treinamento_clean['estado_onde_reside'] = dados_treinamento_clean['estado_onde_reside'].map({estado: indice for indice, estado in enumerate(estados)})
dados_treinamento_clean['estado_onde_nasceu'] = dados_treinamento_clean['estado_onde_nasceu'].map({estado: indice for indice, estado in enumerate(estados)})
dados_treinamento_clean['estado_onde_trabalha'] = dados_treinamento_clean['estado_onde_trabalha'].map({estado: indice for indice, estado in enumerate(estados)})

dados_treinamento_clean['codigo_area_telefone_trabalho'] = dados_treinamento_clean['codigo_area_telefone_trabalho'].replace(' ', 0).astype(np.int64)
dados_treinamento_clean['codigo_area_telefone_residencial'] = dados_treinamento_clean['codigo_area_telefone_residencial'].replace(' ', 0).astype(np.int64)

Faz uma verificação para que todos as colunas possuam apenas elementos númericos, e converte todos os valores para int64, o que será necessário para calcular o valor do coeficiente de Pearson em seguida.

In [7]:
colunas_treinamento = dados_treinamento_clean.columns
colunas_str = []
for col in colunas_treinamento:
    if type(dados_treinamento_clean[col][0]) == str:
        colunas_str.append(col)
    elif dados_treinamento_clean[col][0].dtype != 'int64':
        dados_treinamento_clean[col] = np.int64(dados_treinamento_clean[col])


C:\Users\sergi\anaconda3\Lib\site-packages\pandas\core\series.py:893: RuntimeWarning: invalid value encountered in cast
  return np.asarray(self._values, dtype)


Remove as colunas, que, quando comparadas com o valor target (inadimplente), possuem coeficiente de Pearson menor do que 0.01 em módulo ou igual a NaN (o que ocorre em casos em que todos os valores de uma coluna são iguais).

In [8]:
for col in colunas_treinamento:
    coef_pears = pearsonr(dados_treinamento_clean[col],dados_treinamento_clean['inadimplente'])[0]
    if np.isnan(coef_pears):
        dados_treinamento_clean.drop(columns=col, inplace=True)
    if (coef_pears < 0.005 and coef_pears > -0.005):
        dados_treinamento_clean.drop(columns=col, inplace=True)

Exibe o coeficiente de Pearson de cada coluna restante na tabela (deixei comentado pois era apenas ilustrativo e o código demora um pouco para rodar)

In [9]:
# colunas_treinamento = dados_treinamento_clean.columns
# for col in colunas_treinamento:
#     print('%20s %6.3f' % ( col , pearsonr(dados_treinamento_clean[col],dados_treinamento_clean['inadimplente'])[0] ) )

  produto_solicitado  0.030
      dia_vencimento  0.080
               idade -0.121
        estado_civil -0.031
    qtde_dependentes  0.018
  estado_onde_reside -0.028
codigo_area_telefone_residencial -0.039
         renda_extra  0.006
possui_cartao_mastercard -0.020
qtde_contas_bancarias  0.014
qtde_contas_bancarias_especiais  0.014
        possui_carro  0.013
estado_onde_trabalha  0.028
codigo_area_telefone_trabalho  0.019
            ocupacao -0.009
   local_onde_reside -0.019
 local_onde_trabalha -0.019
        inadimplente  1.000
possui_telefone_residencial_N  0.081
possui_telefone_residencial_Y -0.081
              sexo_   0.008
              sexo_F -0.030
              sexo_M  0.029
possui_telefone_trabalho_N -0.016
possui_telefone_trabalho_Y  0.016
vinculo_formal_com_empresa_N -0.006
vinculo_formal_com_empresa_Y  0.006
forma_envio_solicitacao_correio -0.017
forma_envio_solicitacao_internet -0.006
forma_envio_solicitacao_presencial  0.014


Analisa os coeficientes de Pearson das colunas e retorna eles ordenados pelo valor seu valor em módulo. Ao final remove uma das colunas relacionadas ao telefone residencial por estar redundante no modelo.

In [10]:
colunas_treinamento = dados_treinamento_clean.columns
d = {'coluna':[], 'pearsonr':[]}
pearson_r = pd.DataFrame(data=d)

for col in colunas_treinamento:
    temp_pears = abs(pearsonr(dados_treinamento_clean[col],dados_treinamento_clean['inadimplente'])[0])
    tempDf = pd.DataFrame({'coluna':[col], 'pearsonr':[temp_pears]})
    pearson_r = pd.concat([pearson_r, tempDf], ignore_index=True)

# dados_treinamento_clean.drop(columns=['possui_telefone_residencial_Y', 'qtde_contas_bancarias',
#                                       'possui_telefone_trabalho_N', 'local_onde_trabalha'], inplace=True)    
pearson_r.sort_values(by='pearsonr')

,coluna,pearsonr
7,renda_extra,0.005851
26,vinculo_formal_com_empresa_Y,0.006243
25,vinculo_formal_com_empresa_N,0.006243
28,forma_envio_solicitacao_internet,0.006250
20,sexo_,0.008019
14,ocupacao,0.008989
11,possui_carro,0.012780
29,forma_envio_solicitacao_presencial,0.013617
9,qtde_contas_bancarias,0.014133
10,qtde_contas_bancarias_especiais,0.014133


Cria dois arrays ('x' e 'y'), de forma que o segundo possui apenas a coluna target (inadimplente) e o primeiro possui todas as outras. Em seguida, usa-se a função train_test_split para separar os arrays de treino e os de teste, que serão usados no modelo, com a proporção de 85% e 15%, respectivamente.

In [11]:
from sklearn.model_selection import train_test_split

x = dados_treinamento_clean.drop(columns='inadimplente').values
y = dados_treinamento_clean['inadimplente'].values

x_treino, x_teste, y_treino, y_teste = train_test_split(
        x,
        y,
        test_size =4000,
        random_state = 23
)

Cria um scaler para normalizar os dados dos arrays de treinamento e de teste, de forma que nenhuma coluna possua um peso maior do que a outra a priori.

In [12]:
from sklearn.preprocessing import StandardScaler

scaler  = StandardScaler()
x_treino = scaler.fit_transform(x_treino)
x_teste  = scaler.transform(x_teste)

Aplicação do modelo KNN de previsão, com um range inicial de 100 a 400 vizinhos, que vai se ajustando por meio da função GridSearchCV, até chegar no valor final exato de vizinhos que maximiza a precisão do modelo.

In [13]:
param_grid = {'n_neighbors': range(340, 400, 1)}
knn = KNeighborsClassifier(weights='distance')
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_treino, y_treino)
melhores_parametros = grid_search.best_params_
melhor_acuracia = grid_search.best_score_

print ("K = %i" % (melhores_parametros['n_neighbors']))
print ("Acurácia  = %4.4f %%" % (100*melhor_acuracia))
print ("Taxa Erro = %4.4f %%" % (100*(1-melhor_acuracia)))
print ("\n\n")

K = 384
Acurácia  = 57.4062 %
Taxa Erro = 42.5938 %





Realiza o tratamento dos dados de teste, de forma que ele possua as mesmas colunas dos dados de treinamento.

In [14]:
dados_teste_clean = pd.get_dummies(dados_teste, columns=colunas_texto)
dados_teste_clean.drop(columns=colunas_removidas, inplace=True)

dados_teste_clean['estado_onde_reside'] = dados_teste_clean['estado_onde_reside'].map({estado: indice for indice, estado in enumerate(estados)})
dados_teste_clean['estado_onde_nasceu'] = dados_teste_clean['estado_onde_nasceu'].map({estado: indice for indice, estado in enumerate(estados)})
dados_teste_clean['estado_onde_trabalha'] = dados_teste_clean['estado_onde_trabalha'].map({estado: indice for indice, estado in enumerate(estados)})

dados_teste_clean['codigo_area_telefone_trabalho'] = dados_teste_clean['codigo_area_telefone_trabalho'].replace(' ', 0).astype(np.int64)
dados_teste_clean['codigo_area_telefone_residencial'] = dados_teste_clean['codigo_area_telefone_residencial'].replace(' ', 0).astype(np.int64)

colunas_teste = dados_teste_clean.columns
for col in colunas_teste:
    if type(dados_teste_clean[col][0]) == str:
        colunas_str.append(col)
    elif dados_teste_clean[col][0].dtype != ' int64':
        dados_teste_clean[col] = np.int64(dados_teste_clean[col])


colunas_retiradas_clean = [x for x in colunas_teste if x not in dados_treinamento_clean]
dados_teste_clean.drop(columns=colunas_retiradas_clean, inplace=True)

colunas_ausentes = set(dados_treinamento_clean.columns) - set(dados_teste_clean.columns)
colunas_ausentes.remove('inadimplente')
dados_teste_clean = pd.concat([dados_teste_clean, pd.DataFrame({coluna: [0] * len(dados_teste_clean) for coluna in colunas_ausentes})], axis=1)

C:\Users\sergi\AppData\Local\Temp\ipykernel_21388\735316100.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dados_teste_clean = pd.get_dummies(dados_teste, columns=colunas_texto)
C:\Users\sergi\anaconda3\Lib\site-packages\pandas\core\series.py:893: RuntimeWarning: invalid value encountered in cast
  return np.asarray(self._values, dtype)


Cria o classificador com o maior valor de k encontrado anteriormente, treina-o com os arrays de teste e aplica o scaler nos dados recebidos pelo problema

In [15]:
classificador_final = KNeighborsClassifier(n_neighbors=melhores_parametros['n_neighbors'], weights='distance')

classificador_final = classificador_final.fit(x_treino, y_treino)
x_teste_real = dados_teste_clean.iloc[:, :].to_numpy()
x_teste_real = scaler.transform(x_teste_real)

Aplica o classificador para predizer os dados recebidos e gera o arquivo com as respostas encontradas.

In [16]:
y_resposta_teste_real = classificador_final.predict(x_teste_real)

criar_resposta(y_resposta_teste_real)